In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
yf.pdr_override()
from pandas_datareader import data as pdr
import pickle as pkl
import talib as ta
from time import *
import datetime
from datetime import date 
import pyttsx3
engine = pyttsx3.init()
voices = engine.getProperty('voices')       #getting details of current voice
#engine.setProperty('voice', voices[0].id)  #changing index, changes voices. o for male
engine.setProperty('voice', voices[1].id)   #changing index, changes voices. 1 for female
newVoiceRate = 160
engine.setProperty('rate',newVoiceRate)
engine.say('Hello Sir. Welcome to stock screening')
engine.runAndWait()

## real code run here

### data downloading

In [2]:
def get_data(timefra,companies,date):    
    
    '''return data of whole companies'''
    timeframes = [timefra]
    
    for time in timeframes:
        # download dataframe
        dataset = pdr.get_data_yahoo(companies, 
                                  start = date,
                                  #end="2017-04-30",
                                  interval = time,
                                  group_by ='ticker'
                                 )
    return dataset

In [3]:

def check_rsi(data,companies):
    '''
    return stocks with rsi in overbought and oversold in order
    data = data of all companies which group by ticker
    companies = list of companies in order of data companies
    
    return:
    rsi_overbought_companies,rsi_oversold_companies   
    
    '''
    div_bought_companies = {}
    div_sold_companies = {}
    for company in companies:
        divergence = ta.RSI(data[company]['Close'],timeperiod = 14)
        #data.loc.__setitem__((slice(None), (company, 'Divergence')), divergence)
        rsi = np.array([ divergence[-2],divergence[-3] ])
        if np.max(rsi) >= 70 :
            div_bought_companies[company] = np.max(rsi)
        elif np.min(rsi) <= 30:
            div_sold_companies[company] = np.min(rsi)
        else:
            continue
    div_bought_companies = sorted( div_bought_companies.items(), key=lambda  v: v[1] , reverse=True)    
    div_sold_companies =   sorted( div_sold_companies.items(), key=lambda  v: v[1] )
    return div_bought_companies,div_sold_companies
    

In [4]:
def start(companies , date,data):
    global rsi_overbought_companies,rsi_oversold_companies
    
    #1st turn
    engine.say('Screening started')
    engine.runAndWait()
    dataset = get_data('5m',companies,date)
    #live dataset cleaning....
    array = dataset.index.time.copy()
    seconds = lambda t: int(t.strftime("%H:%M:%S")[-2:])
    minutes = lambda x: int(x.strftime("%H:%M:%S")[-5:-3])
    minute = np.array([minutes(xi) for xi in array])
    dataset = dataset[minute%5 == 0]
    array = dataset.index.time.copy()
    second = np.array([seconds(xi) for xi in array])
    dataset = dataset[second==0]
            
    rsi_overbought_companies,rsi_oversold_companies = check_rsi(dataset , companies)
    #print(rsi_overbought_companies)
    if(len(rsi_overbought_companies)  or len(rsi_oversold_companies) ):
                                                                
        %store rsi_overbought_companies
        %store rsi_oversold_companies
        engine.say('Sir. Companies detected')
        engine.runAndWait()
        
        print("\n --",strftime("%H:%M:%S", localtime()),"---\n")
        
        print(" ============= \n","rsi_overbought_companies\n" )
        print(*rsi_overbought_companies, sep="\n")
        print("\n rsi_oversold_companies \n" )
        print(*rsi_oversold_companies,sep = "\n")
        print("\n ===========\n")
        
        
    # next turns
                                                          
    while True:      
        %store -r var
        if var:
            engine.say('Screening started')
            engine.runAndWait()
            dataset = get_data('5m',companies,date)
            #live dataset cleaning....
            array = dataset.index.time.copy()
            seconds = lambda t: int(t.strftime("%H:%M:%S")[-2:])
            minutes = lambda x: int(x.strftime("%H:%M:%S")[-5:-3])
            minute = np.array([minutes(xi) for xi in array])
            dataset = dataset[minute%5 == 0]
            array = dataset.index.time.copy()
            second = np.array([seconds(xi) for xi in array])
            dataset = dataset[second==0]
            
            rsi_overbought_companies,rsi_oversold_companies = check_rsi(dataset , companies)
            if(len(rsi_overbought_companies) or len(rsi_oversold_companies) ):
                %store rsi_overbought_companies
                %store rsi_oversold_companies
                engine.say('companies detected')
                engine.runAndWait()
        
                print("\n --",strftime("%H:%M:%S", localtime()),"---\n")
            
                print(" ============= \n","rsi_overbought_companies\n" )
                print(*rsi_overbought_companies, sep="\n")
                print("\n rsi_oversold_companies \n" )
                print(*rsi_oversold_companies,sep = "\n")
                print("\n ===========\n")
            else:
                engine.say('No companies are detected')
                engine.runAndWait()
            %store -r var
            print(var)
            sleep((3)*60) 
            print(var)

In [5]:
data = pd.read_csv('ind_nifty50list.csv')
symbols = data['Symbol']
companies = list(symbols+".NS")
timeframes = ['15m']
today = datetime.date.today()
date  = ( today - datetime.timedelta(days = 1) ).strftime("%Y-%m-%d")    


In [6]:
adder = lambda x: x+".NS"
companies = np.array(['HDFC','UPL','TITAN','TECHM','TATASTEEL','TATAMOTORS','SHREECEM','SBILIFE','RELIANCE','ONGC','M&M',
         'LT','IOC','ITC','JSWSTEEL','KOTAKBANK','BAJAJ-AUTO','BAJAJFINSV','BPCL','BHARTIARTL','BRITANNIA',
         'CIPLA','COALINDIA','EICHERMOT','GAIL','GRASIM','HCLTECH','HDFCLIFE','IOC','HINDALCO','HINDUNILVR',
          'HDFC'])
companies = [adder(i) for i in companies]

In [ ]:
start(companies,date,data)

[*********************100%***********************]  30 of 30 completed
Stored 'rsi_overbought_companies' (list)
Stored 'rsi_oversold_companies' (list)

 -- 09:41:43 ---

 rsi_overbought_companies

('COALINDIA.NS', 80.98665491970291)
('BRITANNIA.NS', 76.401254759753)
('JSWSTEEL.NS', 71.72492657462978)
('EICHERMOT.NS', 70.9311651201026)
('IOC.NS', 70.61737627507826)

 rsi_oversold_companies 

('HINDUNILVR.NS', 28.669821945892295)


[*********************100%***********************]  30 of 30 completed
Stored 'rsi_overbought_companies' (list)
Stored 'rsi_oversold_companies' (list)

 -- 09:45:26 ---

 rsi_overbought_companies

('COALINDIA.NS', 80.98665491970291)
('BRITANNIA.NS', 76.401254759753)
('JSWSTEEL.NS', 71.72492657462978)
('EICHERMOT.NS', 70.9311651201026)
('IOC.NS', 70.61737627507826)

 rsi_oversold_companies 

('HINDUNILVR.NS', 28.669821945892295)


False
False
[*********************100%***********************]  30 of 30 completed
Stored 'rsi_overbought_companies' (list)
Stored '